In [2]:
import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
#JupyterDash.infer_jupyter_proxy_config()

In [3]:
y_df = pd.read_csv('../../data/product_output_sample.csv')

In [4]:
y_df

,predict,probs,group,status,subscriber_id,email,intervention_cluster,minutes_watched
0,0.0,0.000000,weekly-cancel,One Week Trial - Recent Cancel,217749.0,utWdEAH@gmail.com,Medium Probability P1,1346
1,0.0,0.000000,weekly-cancel,One Week Trial - Recent Cancel,81316.0,BkSAoLq@gmail.com,Medium Probability P1,1983
2,0.0,0.000000,weekly-cancel,One Week Trial - Recent Cancel,183152.0,uagJZPM@gmail.com,Medium Probability P1,1069
3,0.0,0.000000,weekly-cancel,One Week Trial - Recent Cancel,19137.0,PfNVBeR@gmail.com,Medium Probability P1,3460
4,0.0,0.000000,weekly-cancel,One Week Trial - Recent Cancel,30554.0,QBBfNKA@gmail.com,Medium Probability P1,2642
...,...,...,...,...,...,...,...,...
249973,0.0,0.398317,monthly-eligible,One Month Trial - Medium P1 Probability,224185.0,mSRuwkl@gmail.com,Medium Probability P1,1233
249974,0.0,0.440003,monthly-eligible,One Month Trial - Medium P1 Probability,144294.0,FXmgpAb@gmail.com,Medium Probability P1,123
249975,0.0,0.202035,monthly-eligible,One Month Trial - Low P1 Probability,17919.0,zePhvNs@gmail.com,Low Probability P1: Dormant,114
249976,0.0,0.352113,monthly-eligible,One Month Trial - Medium P1 Probability,65535.0,xrVNiMp@gmail.com,Medium Probability P1,2828


In [7]:
min_watched_avg_df = y_df.groupby(by='intervention_cluster').agg(
    avgminwatched=pd.NamedAgg(column='minutes_watched', aggfunc=np.mean)).reset_index()

In [8]:
min_watched_avg_df

,intervention_cluster,avgminwatched
0,Converter Look-a-Like,1792.516695
1,High Probability P1: 60 Min Usage,1807.881429
2,Low Probability P1: Dormant,1803.174695
3,Low Usage: Difficult Prediction,1798.291209
4,Medium Probability P1,1799.704166
5,No Usage,1802.856544


In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

available_clusters = list(min_watched_avg_df.intervention_cluster.unique())

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_clusters],
                value='No Usage' # the default
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(intervention_cluster):
    filtered_score = min_watched_avg_df[(min_watched_avg_df.intervention_cluster == intervention_cluster)]
    figure = px.bar(filtered_score, x = 'intervention_cluster', y = 'avgminwatched')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)